# Sample MERRA-2 Along a CALIPSO Obrit
This is an example showing how to use GMAOpyobs utilities to sample MERRA-2 along a CALIPSO orbit.

This assumes that you have cloned GMAOpyobs Github repository, installed it in a directory called \\$AERODIR, and have added the following directory to your $PYTHONPATh environment variable:
\\$AERODIR/install/lib/Python

It also assumes you have a grads style control file for the MERRA-2 files. An example of such a file is in this directory called inst3_3d_aer_Nv, and it points to files in a directory called MERRA2_all in this directory.

This MERRA-2 Collection contains 3-D aerosol mass mixing ratios


## CALIPSO Example

In [1]:
import sys
sys.path.append('/gpfsm/dnb34/pcastell/workspace/GMAOpyobs_ipynb_examples/GMAOpyobs/install/lib/Python')

In [3]:
from pyobs.sampler import TRAJECTORY
from pyobs.calipso_l2 import CALIPSO_L2

In [4]:
# set up some file names
calipsoFile = 'CAL_LID_L2_05kmAPro-Standard-V4-51.2016-08-17T05-02-55ZD.hdf'
m2data = 'inst3_3d_aer_Nv'
outFile = 'm2_calipso_sampled.nc4'

In [5]:
# Read the CALIPSO file, get the lat, lon, times from the file
caldata = CALIPSO_L2(calipsoFile)

v Latitude
v Longitude
v Pressure
v Temperature
v Total_Backscatter_Coefficient_532
v Total_Backscatter_Coefficient_Uncertainty_532
v Extinction_Coefficient_532
v Extinction_Coefficient_Uncertainty_532
shape Latitude (4208, 3)
shape Longitude (4208, 3)
shape Profile_UTC_Time (4208, 3)
shape Pressure (4208, 399)
shape Temperature (4208, 399)
shape Total_Backscatter_Coefficient_532 (4208, 399)
shape Total_Backscatter_Coefficient_Uncertainty_532 (4208, 399)
shape Extinction_Coefficient_532 (4208, 399)
shape Extinction_Coefficient_Uncertainty_532 (4208, 399)
shape nymd (17707264,)
shape nhms (17707264,)


In [9]:
# create a trajectory oject
time, lon, lat = caldata.time, caldata.lon[:,1], caldata.lat[:,1]
traj = TRAJECTORY(time,lon,lat,m2data)

In [10]:
# sample the MERRA-2 dataset along the trajectory, and return an xarray dataset
traj_ds = traj.sample()

In [11]:
# write data to a netcdf file
traj_ds.to_netcdf(outFile)

# Calculate Aerosol Optical Properties on the Sampled Dataset
Now that we have the 3-D aerosol mass mixing ratios, we would like to convert this to 3-D total and speciated aerosol optical properties.
You will need the aerosol optics tables, and a yaml file describing which files to use.
An example yaml file is in this directory called Aod_EOS.m2.yaml
It points to optics files located in the directory ExtData in this directory.


In [2]:
from pyobs.aop import G2GAOP

In [3]:
# set up some filenames
configName = 'Aod_EOS.m2.yaml'
aerFile = 'm2_calipso_sampled.nc4'

In [4]:
# read the aerosol data and optical tables
config = open(configName)
optics = G2GAOP(aerFile,config=config)

In [5]:
# caluclate profiles of total:
#        EXT:     aerosol extinction profile
#        SCA:     aerosol scattering profile
#        BSC:     aerosol backscatter profile
#        DEPOL:   aerosol depolarization ratio
# at 532 nm
ext532 = optics.getAOPext(wavelength=532)

In [27]:
# write this to a file
outFile = 'm2_calipso_ext532.nc4'
ext532.to_netcdf(outFile)

In [12]:
# calculate profile of EXT, SCA, BSC, and DEPOL for a each species individually at 532 nm
ee = []
ext532_du = optics.getAOPext(Species='DU',wavelength=532)  # dust
ee.append(ext532_du)
ext532_su = optics.getAOPext(Species='SU',wavelength=532)  # sulfate
ee.append(ext532_su)
ext532_ss = optics.getAOPext(Species='SS',wavelength=532)  # sea salt
ee.append(ext532_ss)
ext532_cc = optics.getAOPext(Species=['OC','BC'],wavelength=532) # organic carbon plus black carbon
ee.append(ext532_cc)

In [13]:
# write these to file
SPC = ['DU','SU','SS','CC']
for spc,ext in zip(SPC,ee):
    outFile = 'm2_calipso_ext532_{}.nc4'.format(spc)
    ext.to_netcdf(outFile)

In [14]:
# calculate profiles of total:
#       AOT: aerosol layer optocal thickness
#       SSA: single scattering albedo
#       G: assymetry parameter
rt532 = optics.getAOPrt(wavelength=532)

# per species
rt532_du = optics.getAOPrt(Species='DU',wavelength=532)

In [15]:
# write these to file
outFile = 'm2_calipso_rt532.nc4'
rt532.to_netcdf(outFile)